In [1]:
from pymongo import MongoClient
from pandas.io.json import json_normalize
import pandas as pd

In [2]:
client = MongoClient ('localhost', 27017)
db = client['companies']

In [3]:
#Listado de categorias de las compañias
categorias = db.companies.distinct('category_code');
list(categorias)

['enterprise',
 'web',
 'software',
 'news',
 'network_hosting',
 'games_video',
 'mobile',
 'music',
 'social',
 'search',
 'messaging',
 'advertising',
 'photo_video',
 'security',
 'finance',
 'ecommerce',
 'travel',
 'hardware',
 'public_relations',
 'other',
 'real_estate',
 'semiconductor',
 'analytics',
 'health',
 'legal',
 'sports',
 'biotech',
 'cleantech',
 'education',
 'consulting',
 'transportation',
 None,
 'hospitality',
 'fashion',
 'nonprofit',
 'nanotech',
 'automotive',
 'design',
 'manufacturing',
 'government',
 'local',
 'medical']

In [4]:
categorias = ['web', 'software', 'games_video', 'social', 'design', 'search']

def buscar_startup(categorias):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$lt':100}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},
                                       {'founded_year':{'$gte': 2010}}]},
                              {'name': 1,
                               'number_of_employees': 1,
                               'category_code': 1,
                               'acquisition': 1,
                               'founded_year': 1,
                               'funding_rounds': 1,
                               'offices.latitude': 1,
                               'offices.longitude': 1})
    return datos

def buscar_big(categorias):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$gt': 1000}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},                                       
                                       {'founded_year':{'$gte': 1990}}]},
                              {'name': 1,
                               'number_of_employees': 1,
                               'category_code': 1,
                               'acquisition': 1,
                               'founded_year': 1,
                               'funding_rounds': 1,
                               'offices.latitude': 1,
                               'offices.longitude': 1,
                               'offices.city': 1,
                               'offices.country_code': 1})
    return datos

#'offices.city': 1,
#'offices.country_code': 1

startup = pd.DataFrame(buscar_startup(categorias))
big_companie = pd.DataFrame(buscar_big(categorias))

#data = pd.concat([startup, big_companie])
#data = data.reset_index()
big_companie.head(100)

,_id,acquisition,category_code,founded_year,funding_rounds,name,number_of_employees,offices
0,52cdef7c4bab8bd675297d8e,None,social,2004,"[{'id': 2, 'round_code': 'angel', 'source_url'...",Facebook,5299,"[{'city': 'Menlo Park', 'country_code': 'USA',..."
1,52cdef7c4bab8bd675297d94,None,social,2006,"[{'id': 261, 'round_code': 'a', 'source_url': ...",Twitter,1300,"[{'city': 'San Francisco', 'country_code': 'US..."
2,52cdef7c4bab8bd675297d9b,None,web,1995,"[{'id': 24534, 'round_code': 'unattributed', '...",eBay,15000,"[{'city': 'San Jose', 'country_code': 'USA', '..."
3,52cdef7c4bab8bd675297da3,None,web,1994,"[{'id': 24531, 'round_code': 'a', 'source_url'...",Yahoo!,13600,"[{'city': 'Sunnyvale', 'country_code': 'USA', ..."
4,52cdef7c4bab8bd675297e0c,None,social,2003,"[{'id': 152, 'round_code': 'a', 'source_url': ...",LinkedIn,4800,"[{'city': 'Mountain View', 'country_code': 'US..."
5,52cdef7c4bab8bd675297dba,None,search,1998,"[{'id': 380, 'round_code': 'angel', 'source_ur...",Google,28000,"[{'city': 'Mountain View', 'country_code': 'US..."
6,52cdef7c4bab8bd675297ea4,None,web,2005,[],Webkinz,8657,"[{'city': 'Woodbridge', 'country_code': 'CAN',..."
7,52cdef7c4bab8bd675297efe,"{'price_amount': 1200000000, 'price_currency_c...",web,1999,"[{'id': 450, 'round_code': 'd', 'source_url': ...",Zappos,1500,"[{'city': 'Henderson', 'country_code': 'USA', ..."
8,52cdef7c4bab8bd675297fcb,None,web,1997,"[{'id': 52008, 'round_code': 'unattributed', '...",Rakuten,10000,"[{'city': 'Tokyo', 'country_code': 'JPN', 'lat..."
9,52cdef7c4bab8bd6752981b5,None,software,1992,[],NetApp,8000,"[{'city': None, 'country_code': 'USA', 'latitu..."


In [148]:
startup.shape

(32, 8)

In [7]:
big_office_data = json_normalize(data = buscar_big(categorias), 
                             record_path ='offices', 
                             meta=['_id', 'name', 'category_code', 'founded_year', 'number_of_employees']) 
big_office_data

,city,country_code,latitude,longitude,_id,name,category_code,founded_year,number_of_employees
0,Menlo Park,USA,37.416050,-122.151801,52cdef7c4bab8bd675297d8e,Facebook,social,2004,5299
1,Dublin,IRL,53.344104,-6.267494,52cdef7c4bab8bd675297d8e,Facebook,social,2004,5299
2,New York,USA,40.755716,-73.979247,52cdef7c4bab8bd675297d8e,Facebook,social,2004,5299
3,San Francisco,USA,37.776805,-122.416924,52cdef7c4bab8bd675297d94,Twitter,social,2006,1300
4,San Jose,USA,37.295005,-121.930035,52cdef7c4bab8bd675297d9b,eBay,web,1995,15000
5,Sunnyvale,USA,37.418531,-122.025485,52cdef7c4bab8bd675297da3,Yahoo!,web,1994,13600
6,Mountain View,USA,37.423258,-122.073635,52cdef7c4bab8bd675297e0c,LinkedIn,social,2003,4800
7,Mountain View,USA,37.421972,-122.084143,52cdef7c4bab8bd675297dba,Google,search,1998,28000
8,Ann Arbor,USA,42.280988,-83.748882,52cdef7c4bab8bd675297dba,Google,search,1998,28000
9,Atlanta,USA,33.781466,-84.387519,52cdef7c4bab8bd675297dba,Google,search,1998,28000


In [8]:
startup_office_data = json_normalize(data = buscar_startup(categorias), 
                             record_path ='offices', 
                             meta=['_id', 'name', 'category_code', 'founded_year', 'number_of_employees']) 
startup_office_data

,latitude,longitude,_id,name,category_code,founded_year,number_of_employees
0,40.757929,-73.985506,52cdef7c4bab8bd675297f94,PeekYou,search,2012,20
1,52.501345,13.410907,52cdef7c4bab8bd675297fb1,headr,web,2012,8
2,NaN,NaN,52cdef7c4bab8bd675297fb1,headr,web,2012,8
3,37.566879,-122.323895,52cdef7c4bab8bd675297fec,Fixya,web,2013,30
4,NaN,NaN,52cdef7c4bab8bd675298523,Simplicant,software,2012,10
5,37.760524,-122.387799,52cdef7d4bab8bd675298a4d,Fuzz,games_video,2011,6
6,40.650291,-74.294395,52cdef7d4bab8bd675298cff,CollegeConvo,web,2010,2
7,38.989124,-77.026676,52cdef7d4bab8bd675298da5,Carfeine,software,2012,5
8,37.444098,-122.161287,52cdef7d4bab8bd675298dba,Ziippi,web,2011,6
9,53.544711,-113.515769,52cdef7d4bab8bd675298dcc,Titan Gaming,games_video,2010,18
